Install dependency

In [1]:
%cd /home/bochao/
!pip install tensorflow-gan

/home/bochao


Compute Inception Score using tensorflow.gan

In [25]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import os
import functools
import numpy as np
import time
from tensorflow.python.ops import array_ops
# pip install tensorflow-gan
import tensorflow_gan as tfgan
session=tf.compat.v1.InteractiveSession()
# A smaller BATCH_SIZE reduces GPU memory usage, but at the cost of a slight slowdown
BATCH_SIZE = 1
INCEPTION_TFHUB = 'https://tfhub.dev/tensorflow/tfgan/eval/inception/1'
INCEPTION_OUTPUT = 'logits'

# Run images through Inception.
inception_images = tf.compat.v1.placeholder(tf.float32, [None, 3, None, None], name = 'inception_images')
def inception_logits(images = inception_images, num_splits = 1):
    images = tf.transpose(images, [0, 2, 3, 1])
    size = 299
    images = tf.compat.v1.image.resize_bilinear(images, [size, size])
    generated_images_list = array_ops.split(images, num_or_size_splits = num_splits)
    logits = tf.map_fn(
        fn = tfgan.eval.classifier_fn_from_tfhub(INCEPTION_TFHUB, INCEPTION_OUTPUT, True),
        elems = array_ops.stack(generated_images_list),
        parallel_iterations = 8,
        back_prop = False,
        swap_memory = True,
        name = 'RunClassifier')
    logits = array_ops.concat(array_ops.unstack(logits), 0)
    return logits

logits=inception_logits()

def get_inception_probs(inps):
    session=tf.get_default_session()
    n_batches = int(np.ceil(float(inps.shape[0]) / BATCH_SIZE))
    preds = np.zeros([inps.shape[0], 1000], dtype = np.float32)
    for i in range(n_batches):
        inp = (inps[i * BATCH_SIZE:(i + 1) * BATCH_SIZE] / 255. * 2 - 1).astype(np.float32)
        preds[i * BATCH_SIZE : i * BATCH_SIZE + min(BATCH_SIZE, inp.shape[0])] = session.run(logits,{inception_images: inp})[:, :1000]
    preds = np.exp(preds) / np.sum(np.exp(preds), 1, keepdims=True)
    return preds

def preds2score(preds, splits=10):
    scores = []
    for i in range(splits):
        part = preds[(i * preds.shape[0] // splits):((i + 1) * preds.shape[0] // splits), :]
        kl = part * (np.log(part) - np.log(np.expand_dims(np.mean(part, 0), 0)))
        kl = np.mean(np.sum(kl, 1))
        scores.append(np.exp(kl))
    return np.mean(scores), np.std(scores)

def get_inception_score(images, splits=10):
    assert(type(images) == np.ndarray)
    assert(len(images.shape) == 4)
    assert(images.shape[1] == 3)
    assert(np.min(images[0]) >= 0 and np.max(images[0]) > 10), 'Image values should be in the range [0, 255]'
    print('Calculating Inception Score with %i images in %i splits' % (images.shape[0], splits))
    start_time=time.time()
    preds = get_inception_probs(images)
    mean, std = preds2score(preds, splits)
    print('Inception Score calculation time: %f s' % (time.time() - start_time))
    return mean, std  # Reference values: 11.38 for 50000 CIFAR-10 training set images, or mean=11.31, std=0.10 if in 10 splits.


Compute FID using tensorflow.gan

In [30]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import os
import functools
import numpy as np
import time
from tensorflow.python.ops import array_ops
# pip install tensorflow-gan
import tensorflow_gan as tfgan

session=tf.compat.v1.InteractiveSession()
# A smaller BATCH_SIZE reduces GPU memory usage, but at the cost of a slight slowdown
BATCH_SIZE = 1

# Run images through Inception.
inception_images = tf.compat.v1.placeholder(tf.float32, [None, 3, None, None], name = 'inception_images')
activations1 = tf.compat.v1.placeholder(tf.float32, [None, None], name = 'activations1')
activations2 = tf.compat.v1.placeholder(tf.float32, [None, None], name = 'activations2')
fcd = tfgan.eval.frechet_classifier_distance_from_activations(activations1, activations2)

INCEPTION_TFHUB = 'https://tfhub.dev/tensorflow/tfgan/eval/inception/1'
INCEPTION_FINAL_POOL = 'pool_3'

def inception_activations(images = inception_images, num_splits = 1):
    images = tf.transpose(images, [0, 2, 3, 1])
    size = 299
    images = tf.compat.v1.image.resize_bilinear(images, [size, size])
    generated_images_list = array_ops.split(images, num_or_size_splits = num_splits)
    activations = tf.map_fn(
        fn = tfgan.eval.classifier_fn_from_tfhub(INCEPTION_TFHUB, INCEPTION_FINAL_POOL, True),
        elems = array_ops.stack(generated_images_list),
        parallel_iterations = 1,
        back_prop = False,
        swap_memory = True,
        name = 'RunClassifier')
    activations = array_ops.concat(array_ops.unstack(activations), 0)
    return activations

activations =inception_activations()

def get_inception_activations(inps):
    n_batches = int(np.ceil(float(inps.shape[0]) / BATCH_SIZE))
    act = np.zeros([inps.shape[0], 2048], dtype = np.float32)
    for i in range(n_batches):
        inp = (inps[i * BATCH_SIZE : (i + 1) * BATCH_SIZE] / 255. * 2 - 1).astype(np.float32)
        act[i * BATCH_SIZE : i * BATCH_SIZE + min(BATCH_SIZE, inp.shape[0])] = session.run(activations, feed_dict = {inception_images: inp})
    return act

def activations2distance(act1, act2):
    return session.run(fcd, feed_dict = {activations1: act1, activations2: act2})
        
def get_fid(images1, images2):
    session=tf.get_default_session()
    assert(type(images1) == np.ndarray)
    assert(len(images1.shape) == 4)
    assert(images1.shape[1] == 3)
    assert(np.min(images1[0]) >= 0 and np.max(images1[0]) > 10), 'Image values should be in the range [0, 255]'
    assert(type(images2) == np.ndarray)
    assert(len(images2.shape) == 4)
    assert(images2.shape[1] == 3)
    assert(np.min(images2[0]) >= 0 and np.max(images2[0]) > 10), 'Image values should be in the range [0, 255]'
    assert(images1.shape == images2.shape), 'The two numpy arrays must have the same shape'
    print('Calculating FID with %i images from each distribution' % (images1.shape[0]))
    start_time = time.time()
    act1 = get_inception_activations(images1)
    act2 = get_inception_activations(images2)
    fid = activations2distance(act1, act2)
    print('FID calculation time: %f s' % (time.time() - start_time))
    return fid


Instructions for updating:
non-resource variables are not supported in the long term



/opt/conda/lib/python3.7/site-packages/tensorflow/python/util/nest.py:914: UserWarning: `tf.layers.flatten` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Flatten` instead.
  structure[0], [func(*x) for x in entries],
/opt/conda/lib/python3.7/site-packages/keras/legacy_tf_layers/core.py:541: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)


Evaluate IS and FID of the generated image

In [29]:
import cv2
import albumentations
import glob
import tensorflow as tf

# categories = [
#  'jasmine', 'phlox', 'leucan', 'cherry',
#  'viola', 'lily', 'appleTree', 'snowdrop',
#  'perennial', 'blackberry', 'strawberry', 'nankingcherry',
#  'bellflower'
#]
# Change these path to test IS and FID for different images
category = 'jasmine'
TRAIN_DATA_DIR = '/home/bochao/flowers/' + category
GEN_DATA_DIR = '/home/bochao/results/' + category
IMG_SIZE = 256

# define our crop schema for train imgs
rescaler = albumentations.SmallestMaxSize(max_size=256)
cropper = albumentations.CenterCrop(height=256, width=256)
preprocessor = albumentations.Compose([rescaler, cropper])

def read_gen_imgs(dir_path):
    img_paths = glob.glob(dir_path + '/*.png')
    n = len(img_paths)
    gen_imgs = np.zeros([n, IMG_SIZE, IMG_SIZE, 3], dtype=np.uint8)
    for i in range(n):
        path = img_paths[i]
        gen_imgs[i] = cv2.imread(path).astype(np.uint8)
    return gen_imgs   

def read_train_imgs(dir_path):
    img_paths = glob.glob(dir_path + '/*.png')
    n = len(img_paths)
    train_imgs = np.zeros([n, IMG_SIZE, IMG_SIZE, 3], dtype=np.uint8)
    for i in range(n):
        path = img_paths[i]
        img = cv2.imread(path).astype(np.uint8)
        img = preprocessor(image=img)["image"]
        assert (IMG_SIZE, IMG_SIZE, 3) == img.shape
        tf.keras.utils.save_img(f'/home/bochao/crop_flowers/jasmine/{i}.png', x=img, data_format='channels_last')
        train_imgs[i] = img
    return train_imgs


In [18]:
# modify path to train directory and generate image directory
# to find the IS and FID scores

# Baseline tokenizer IS, FID scores
TRAIN_DATA_DIR = '/home/bochao/flowers/jasmine'
GEN_DATA_DIR = '/home/bochao/results/tokenizer_baseline'

# read train and generated images
train_imgs = read_train_imgs(TRAIN_DATA_DIR)
gen_imgs = read_gen_imgs(GEN_DATA_DIR)

# IS scores
is_mean, is_std = get_inception_score(gen_imgs.transpose([0,3,1,2]), splits=10)
print(f'Inception Score: mean={is_mean:.4f}, std={is_std:.4f}')

Calculating Inception Score with 50 images in 10 splits
Inception Score calculation time: 1.812045 s
Inception Score: mean=1.4658, std=0.1932


In [24]:
# FID scores
fid_score = get_fid(gen_imgs.transpose([0,3,1,2]), train_imgs.transpose([0,3,1,2]))
print(f'FID: {fid_score:.4f}')

Calculating FID with 50 images from each distribution
FID calculation time: 8.389242 s
FID: 216.2836


In [26]:
# Baseline MaskGIT IS, FID scores
TRAIN_DATA_DIR = '/home/bochao/flowers/jasmine'
GEN_DATA_DIR = '/home/bochao/results/transformer_baseline'

# read train and generated images
train_imgs = read_train_imgs(TRAIN_DATA_DIR)
gen_imgs = read_gen_imgs(GEN_DATA_DIR)

# IS scores
is_mean, is_std = get_inception_score(gen_imgs.transpose([0,3,1,2]), splits=10)
print(f'Inception Score: mean={is_mean:.4f}, std={is_std:.4f}')

Calculating Inception Score with 1000 images in 10 splits
Inception Score calculation time: 14.996258 s
Inception Score: mean=2.2504, std=0.0649


In [1]:
# FID scores
!python -m pytorch_fid /home/bochao/results/transformer_baseline /home/bochao/crop_flowers/jasmine

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.88it/s]
FID:  234.46482060506816
